Magenta的数据集为tfrecord格式。tfrecord格式为Tensorflow特有的数据格式，能够将数据和标签二进制存储在一起,并实现高效读写。对于tfrecord的更多介绍，见：<https://www.tensorflow.org/tutorials/load_data/tf-records>。

但是，对于tfrecord的读操作需要知道目标数据的数据结构。并且，在读取时，需要用到tf.train.string_input_producer进行队列操作，见：<https://www.tensorflow.org/api_guides/python/reading_data>。如果你认为这种读写数据的方式不利于调试的话（至少我这么认为），这里给出了一个将tfrecord文件转存为npy文件的代码。

In [1]:
import tensorflow as tf
import magenta
import os
import numpy as np

In [2]:
file_list=[r'E:\Magenta\Dataset\melody_rnn_bach\training_melodies.tfrecord',
           r'E:\Magenta\Dataset\melody_rnn_jazz\training_melodies.tfrecord']

sess=tf.Session()
name_list=[r'bach.npy',
           r'jazz.npy']

In [3]:
def count_record(file_list):
    count_list=[]
    for fn in file_list:
        c = 0
        for record in tf.python_io.tf_record_iterator(fn):
            c += 1
        print(c)
        count_list.append(c)
    return count_list

In [4]:
def tfrecord2npy(file,count):
    #The first half of this function is in magenta.common.sequence_example_lib.get_padded_batch
    input_size=38 #The amount of num_class in melody_rnn
    label_shape=[] #Just for compatibility
    file_queue = tf.train.string_input_producer(file)
    reader = tf.TFRecordReader()
    read_queue, serialized_example = reader.read(file_queue)

    sequence_features = {
        'inputs': tf.FixedLenSequenceFeature(shape=[input_size],
                                             dtype=tf.float32),
        'labels': tf.FixedLenSequenceFeature(shape=label_shape or [],
                                             dtype=tf.int64)}

    single_queue, sequence = tf.parse_single_sequence_example(
        serialized_example, sequence_features=sequence_features)
    
    
    data=[]
    
    #https://www.tensorflow.org/api_guides/python/reading_data
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess, coord)
    try:
        for i in range (count):
            seq=sess.run(sequence)
            data.append(seq)
    except tf.errors.OutOfRangeError:
        print("done")
    finally:
        coord.request_stop()
    #coord.join(threads) #There would be some bugs when this line is added
    return data

In [5]:
count_list=count_record(file_list)

1513
1070


In [6]:
for i in range(len(file_list)):
    path=[file_list[i]] 
    count=count_list[i]
    data=tfrecord2npy(path,count)
    np.save(name_list[i],data)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, FIFOQueue '_1_input_producer' is closed.
	 [[{{node input_producer/input_producer_EnqueueMany}} = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer, input_producer/Identity/_9)]]
